In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import h5py
import os
import importlib

from kaggle_submit import submit_to_kaggle


from objects import *
from helpers import *
# from utils.globals import *
from utils.distribution_statistics import *

train_file = "kaggle_data/X_train.h5/X_train.h5"
test_file = "kaggle_data/X_test.h5/X_test.h5"

def get_train_test_connections():
    h5_train = h5py.File(train_file, mode='a')
    h5_test = h5py.File(test_file, mode='a')
    return h5_train, h5_test

def close_train_test_connections(h5_train, h5_test):
    h5_train.close()
    h5_test.close()
    
#h5_train, h5_test = get_train_test_connections()

y_train = pd.read_csv("kaggle_data/y_train.csv", index_col=0, squeeze=True)
y_train_arr = y_train.to_numpy()

from utils.globals import *

h5_train, h5_test = get_train_test_connections()

In [2]:
from helpers import get_subject_ids

train_ids = get_subject_ids(h5_train)
np.random.seed(1)
train_ids = np.random.permutation(train_ids)
train_ids, validation_ids = train_ids[:28], train_ids[28:]


In [3]:
from final_models.best_rf import BestRF

rf = BestRF(h5_train, h5_test, y_train_arr, train_ids)
rf.train()
print(rf.validation_score)

[Parallel(n_jobs=-2)]: Using backend ThreadingBackend with 3 concurrent workers.
[Parallel(n_jobs=-2)]: Done  44 tasks      | elapsed:    8.5s
[Parallel(n_jobs=-2)]: Done 100 out of 100 | elapsed:   19.1s finished
[Parallel(n_jobs=3)]: Using backend ThreadingBackend with 3 concurrent workers.
[Parallel(n_jobs=3)]: Done  44 tasks      | elapsed:    0.0s
[Parallel(n_jobs=3)]: Done 100 out of 100 | elapsed:    0.0s finished


0.7591152255972616


In [4]:
from final_models.best_hgb import BestHGB

hgb = BestHGB(h5_train, h5_test, y_train_arr, train_ids)
hgb.train()
print(hgb.validation_score)

Binning 0.115 GB of training data: 3.253 s
Binning 0.013 GB of validation data: 0.056 s
Fitting gradient boosted rounds:
[1/1000] 5 trees, 155 leaves (31 on avg), max depth = 9, train loss: 1.26780, val loss: 1.28141, in 1.363s
[2/1000] 5 trees, 155 leaves (31 on avg), max depth = 11, train loss: 1.13080, val loss: 1.15616, in 2.393s
[3/1000] 5 trees, 155 leaves (31 on avg), max depth = 9, train loss: 1.02543, val loss: 1.05826, in 1.634s
[4/1000] 5 trees, 155 leaves (31 on avg), max depth = 9, train loss: 0.93966, val loss: 0.98096, in 1.252s
[5/1000] 5 trees, 155 leaves (31 on avg), max depth = 9, train loss: 0.86914, val loss: 0.92067, in 1.238s
[6/1000] 5 trees, 155 leaves (31 on avg), max depth = 10, train loss: 0.80959, val loss: 0.87099, in 1.601s
[7/1000] 5 trees, 155 leaves (31 on avg), max depth = 12, train loss: 0.75788, val loss: 0.82642, in 1.684s
[8/1000] 5 trees, 155 leaves (31 on avg), max depth = 10, train loss: 0.71364, val loss: 0.78840, in 0.939s
[9/1000] 5 trees, 1

[76/1000] 5 trees, 155 leaves (31 on avg), max depth = 12, train loss: 0.10340, val loss: 0.39519, in 0.600s
[77/1000] 5 trees, 155 leaves (31 on avg), max depth = 13, train loss: 0.10114, val loss: 0.39398, in 0.738s
[78/1000] 5 trees, 155 leaves (31 on avg), max depth = 14, train loss: 0.09895, val loss: 0.39365, in 0.726s
[79/1000] 5 trees, 155 leaves (31 on avg), max depth = 13, train loss: 0.09688, val loss: 0.39324, in 1.352s
[80/1000] 5 trees, 155 leaves (31 on avg), max depth = 14, train loss: 0.09483, val loss: 0.39285, in 0.709s
[81/1000] 5 trees, 155 leaves (31 on avg), max depth = 15, train loss: 0.09282, val loss: 0.39213, in 0.756s
[82/1000] 5 trees, 155 leaves (31 on avg), max depth = 16, train loss: 0.09080, val loss: 0.39114, in 0.650s
[83/1000] 5 trees, 155 leaves (31 on avg), max depth = 14, train loss: 0.08896, val loss: 0.39086, in 0.916s
[84/1000] 5 trees, 155 leaves (31 on avg), max depth = 12, train loss: 0.08717, val loss: 0.38947, in 0.628s
[85/1000] 5 trees, 

In [36]:
from final_models.classic_et import ClassicET

et = ClassicET(h5_train, h5_test, y_train_arr, train_ids)
et.train()
print(et.validation_score)

[Parallel(n_jobs=-2)]: Using backend ThreadingBackend with 3 concurrent workers.
[Parallel(n_jobs=-2)]: Done  44 tasks      | elapsed:    2.7s
[Parallel(n_jobs=-2)]: Done 100 out of 100 | elapsed:    6.1s finished
[Parallel(n_jobs=3)]: Using backend ThreadingBackend with 3 concurrent workers.
[Parallel(n_jobs=3)]: Done  44 tasks      | elapsed:    0.0s
[Parallel(n_jobs=3)]: Done 100 out of 100 | elapsed:    0.1s finished


0.7181104483191002


In [44]:
from final_models.classic_bg import ClassicBG

bg = ClassicBG(h5_train, h5_test, y_train_arr, train_ids)
bg.train()
print(bg.validation_score)

[Parallel(n_jobs=3)]: Using backend LokyBackend with 3 concurrent workers.
[Parallel(n_jobs=3)]: Done   3 out of   3 | elapsed:  7.6min finished
[Parallel(n_jobs=3)]: Using backend LokyBackend with 3 concurrent workers.


0.7285854954717148


[Parallel(n_jobs=3)]: Done   3 out of   3 | elapsed:    0.3s finished


In [5]:
from helpers import subjects_ids_to_indexers, custom_score

y_validation_true = y_train_arr[subjects_ids_to_indexers(h5_train, subjects_ids=sorted(validation_ids), as_indices=True)]

from final_models.vote import vote

print("Validation score after vote (hard policy):")
y_validation_pred_hard = vote(models=[hgb, rf], weights=[0.5, 0.5], policy='hard', kind='validation')
custom_score(y_validation_pred_hard, y_validation_true)


Validation score after vote (hard policy):


[Parallel(n_jobs=3)]: Using backend ThreadingBackend with 3 concurrent workers.
[Parallel(n_jobs=3)]: Done  44 tasks      | elapsed:    0.0s
[Parallel(n_jobs=3)]: Done 100 out of 100 | elapsed:    0.1s finished


0.7704166576658918

In [49]:
import itertools

# HGB PREDOMINANCE

hgb_vals = np.linspace(0.6, 0.8, num=5)
rf_vals = np.linspace(0.1, 0.2, num=5)
et_vals = np.linspace(0.01, 0.2, num=5)


scores_hgb_predominance = list()

for i, (hgb_weight, rf_weight, et_weight) in enumerate(itertools.product(hgb_vals, rf_vals, et_vals)):
    print(i+1, '/', len(hgb_vals) * len(rf_vals) * len(et_vals))
    
    bg_weight = 1 - (hgb_weight + rf_weight + et_weight)
    if bg_weight < 0:
        continue
    
    y_validation_pred_soft = vote(
        models=[hgb, rf, et, bg], 
        weights=[hgb_weight, rf_weight, et_weight, bg_weight],
        policy='soft',
        kind='validation'
    )
    score_soft = custom_score(y_validation_pred_soft, y_validation_true)

    y_validation_pred_hard = vote(
        models=[hgb, rf, et, bg], 
        weights=[hgb_weight, rf_weight, et_weight, bg_weight], 
        policy='hard',
        kind='validation'
    )
    score_hard = custom_score(y_validation_pred_hard, y_validation_true)

    scores_hgb_predominance.append(
        (hgb_weight, rf_weight, et_weight, bg_weight, score_soft, score_hard)
    )

1 / 125


[Parallel(n_jobs=3)]: Using backend ThreadingBackend with 3 concurrent workers.
[Parallel(n_jobs=3)]: Done  44 tasks      | elapsed:    0.0s
[Parallel(n_jobs=3)]: Done 100 out of 100 | elapsed:    0.0s finished
[Parallel(n_jobs=3)]: Using backend ThreadingBackend with 3 concurrent workers.
[Parallel(n_jobs=3)]: Done  44 tasks      | elapsed:    0.0s
[Parallel(n_jobs=3)]: Done 100 out of 100 | elapsed:    0.0s finished
[Parallel(n_jobs=3)]: Using backend LokyBackend with 3 concurrent workers.
[Parallel(n_jobs=3)]: Done   3 out of   3 | elapsed:    1.3s finished
[Parallel(n_jobs=3)]: Using backend ThreadingBackend with 3 concurrent workers.
[Parallel(n_jobs=3)]: Done  44 tasks      | elapsed:    0.0s
[Parallel(n_jobs=3)]: Done 100 out of 100 | elapsed:    0.0s finished
[Parallel(n_jobs=3)]: Using backend ThreadingBackend with 3 concurrent workers.
[Parallel(n_jobs=3)]: Done  44 tasks      | elapsed:    0.0s
[Parallel(n_jobs=3)]: Done 100 out of 100 | elapsed:    0.0s finished
[Parallel(n

2 / 125


[Parallel(n_jobs=3)]: Done 100 out of 100 | elapsed:    0.0s finished
[Parallel(n_jobs=3)]: Using backend ThreadingBackend with 3 concurrent workers.
[Parallel(n_jobs=3)]: Done  44 tasks      | elapsed:    0.0s
[Parallel(n_jobs=3)]: Done 100 out of 100 | elapsed:    0.0s finished
[Parallel(n_jobs=3)]: Using backend LokyBackend with 3 concurrent workers.
[Parallel(n_jobs=3)]: Done   3 out of   3 | elapsed:    0.3s finished
[Parallel(n_jobs=3)]: Using backend ThreadingBackend with 3 concurrent workers.
[Parallel(n_jobs=3)]: Done  44 tasks      | elapsed:    0.0s
[Parallel(n_jobs=3)]: Done 100 out of 100 | elapsed:    0.1s finished
[Parallel(n_jobs=3)]: Using backend ThreadingBackend with 3 concurrent workers.
[Parallel(n_jobs=3)]: Done  44 tasks      | elapsed:    0.0s
[Parallel(n_jobs=3)]: Done 100 out of 100 | elapsed:    0.0s finished
[Parallel(n_jobs=3)]: Using backend LokyBackend with 3 concurrent workers.
[Parallel(n_jobs=3)]: Done   3 out of   3 | elapsed:    0.3s finished
[Parall

3 / 125


...

[Parallel(n_jobs=3)]: Done  44 tasks      | elapsed:    0.0s
[Parallel(n_jobs=3)]: Done 100 out of 100 | elapsed:    0.0s finished
[Parallel(n_jobs=3)]: Using backend ThreadingBackend with 3 concurrent workers.
[Parallel(n_jobs=3)]: Done  44 tasks      | elapsed:    0.0s
[Parallel(n_jobs=3)]: Done 100 out of 100 | elapsed:    0.0s finished
[Parallel(n_jobs=3)]: Using backend LokyBackend with 3 concurrent workers.
[Parallel(n_jobs=3)]: Done   3 out of   3 | elapsed:    0.4s finished
[Parallel(n_jobs=3)]: Using backend ThreadingBackend with 3 concurrent workers.
[Parallel(n_jobs=3)]: Done  44 tasks      | elapsed:    0.0s
[Parallel(n_jobs=3)]: Done 100 out of 100 | elapsed:    0.0s finished
[Parallel(n_jobs=3)]: Using backend ThreadingBackend with 3 concurrent workers.
[Parallel(n_jobs=3)]: Done  44 tasks      | elapsed:    0.0s
[Parallel(n_jobs=3)]: Done 100 out of 100 | elapsed:    0.0s finished
[Parallel(n_jobs=3)]: Using backend LokyBackend with 3 concurrent workers.
[Parallel(n_jobs

112 / 125
113 / 125
114 / 125
115 / 125
116 / 125


[Parallel(n_jobs=3)]: Using backend ThreadingBackend with 3 concurrent workers.
[Parallel(n_jobs=3)]: Done  44 tasks      | elapsed:    0.0s
[Parallel(n_jobs=3)]: Done 100 out of 100 | elapsed:    0.0s finished
[Parallel(n_jobs=3)]: Using backend LokyBackend with 3 concurrent workers.
[Parallel(n_jobs=3)]: Done   3 out of   3 | elapsed:    0.3s finished
[Parallel(n_jobs=3)]: Using backend ThreadingBackend with 3 concurrent workers.
[Parallel(n_jobs=3)]: Done  44 tasks      | elapsed:    0.0s
[Parallel(n_jobs=3)]: Done 100 out of 100 | elapsed:    0.0s finished
[Parallel(n_jobs=3)]: Using backend ThreadingBackend with 3 concurrent workers.
[Parallel(n_jobs=3)]: Done  44 tasks      | elapsed:    0.0s
[Parallel(n_jobs=3)]: Done 100 out of 100 | elapsed:    0.1s finished
[Parallel(n_jobs=3)]: Using backend LokyBackend with 3 concurrent workers.


117 / 125
118 / 125
119 / 125
120 / 125
121 / 125
122 / 125
123 / 125
124 / 125
125 / 125


[Parallel(n_jobs=3)]: Done   3 out of   3 | elapsed:    0.4s finished


In [55]:
scores_hgb_pred_df = pd.DataFrame(scores_hgb_predominance, columns=['hgb_weight', 'rf_weight', 'et_weight', 'bg_weight', 'score_soft', 'score_hard'])
scores_hgb_pred_df.sort_values(by=["score_soft"])


,hgb_weight,rf_weight,et_weight,bg_weight,score_soft,score_hard
5,0.60,0.125,0.0100,0.2650,0.796159,0.796073
22,0.60,0.200,0.1050,0.0950,0.796279,0.796073
20,0.60,0.200,0.0100,0.1900,0.796293,0.796073
10,0.60,0.150,0.0100,0.2400,0.796302,0.796073
15,0.60,0.175,0.0100,0.2150,0.796302,0.796073
...,...,...,...,...,...,...
62,0.70,0.200,0.0100,0.0900,0.800769,0.796073
66,0.75,0.100,0.1050,0.0450,0.801174,0.796073
68,0.75,0.125,0.0575,0.0675,0.801174,0.796073
69,0.75,0.125,0.1050,0.0200,0.801174,0.796073


In [51]:
import itertools

# HGB PREDOMINANCE

rf_vals = np.linspace(0.6, 0.8, num=5)
hgb_vals = np.linspace(0.1, 0.2, num=5)
et_vals = np.linspace(0.01, 0.2, num=5)


scores_rf_predominance = list()

for i, (hgb_weight, rf_weight, et_weight) in enumerate(itertools.product(hgb_vals, rf_vals, et_vals)):
    print(i+1, '/', len(hgb_vals) * len(rf_vals) * len(et_vals))
    
    bg_weight = 1 - (hgb_weight + rf_weight + et_weight)
    if bg_weight < 0:
        continue
    
    y_validation_pred_soft = vote(
        models=[hgb, rf, et, bg], 
        weights=[hgb_weight, rf_weight, et_weight, bg_weight],
        policy='soft',
        kind='validation'
    )
    score_soft = custom_score(y_validation_pred_soft, y_validation_true)

    y_validation_pred_hard = vote(
        models=[hgb, rf, et, bg], 
        weights=[hgb_weight, rf_weight, et_weight, bg_weight], 
        policy='hard',
        kind='validation'
    )
    score_hard = custom_score(y_validation_pred_hard, y_validation_true)

    scores_rf_predominance.append(
        (hgb_weight, rf_weight, et_weight, bg_weight, score_soft, score_hard)
    )

1 / 125


[Parallel(n_jobs=3)]: Using backend ThreadingBackend with 3 concurrent workers.
[Parallel(n_jobs=3)]: Done  44 tasks      | elapsed:    0.0s
[Parallel(n_jobs=3)]: Done 100 out of 100 | elapsed:    0.0s finished
[Parallel(n_jobs=3)]: Using backend ThreadingBackend with 3 concurrent workers.
[Parallel(n_jobs=3)]: Done  44 tasks      | elapsed:    0.0s
[Parallel(n_jobs=3)]: Done 100 out of 100 | elapsed:    0.0s finished
[Parallel(n_jobs=3)]: Using backend LokyBackend with 3 concurrent workers.
[Parallel(n_jobs=3)]: Done   3 out of   3 | elapsed:    0.3s finished
[Parallel(n_jobs=3)]: Using backend ThreadingBackend with 3 concurrent workers.
[Parallel(n_jobs=3)]: Done  44 tasks      | elapsed:    0.0s
[Parallel(n_jobs=3)]: Done 100 out of 100 | elapsed:    0.0s finished
[Parallel(n_jobs=3)]: Using backend ThreadingBackend with 3 concurrent workers.
[Parallel(n_jobs=3)]: Done  44 tasks      | elapsed:    0.0s
[Parallel(n_jobs=3)]: Done 100 out of 100 | elapsed:    0.0s finished
[Parallel(n

2 / 125


[Parallel(n_jobs=3)]: Using backend ThreadingBackend with 3 concurrent workers.
[Parallel(n_jobs=3)]: Done  44 tasks      | elapsed:    0.0s
[Parallel(n_jobs=3)]: Done 100 out of 100 | elapsed:    0.1s finished
[Parallel(n_jobs=3)]: Using backend LokyBackend with 3 concurrent workers.
[Parallel(n_jobs=3)]: Done   3 out of   3 | elapsed:    0.3s finished
[Parallel(n_jobs=3)]: Using backend ThreadingBackend with 3 concurrent workers.
[Parallel(n_jobs=3)]: Done  44 tasks      | elapsed:    0.0s
[Parallel(n_jobs=3)]: Done 100 out of 100 | elapsed:    0.0s finished
[Parallel(n_jobs=3)]: Using backend ThreadingBackend with 3 concurrent workers.
[Parallel(n_jobs=3)]: Done  44 tasks      | elapsed:    0.0s
[Parallel(n_jobs=3)]: Done 100 out of 100 | elapsed:    0.0s finished
[Parallel(n_jobs=3)]: Using backend LokyBackend with 3 concurrent workers.
[Parallel(n_jobs=3)]: Done   3 out of   3 | elapsed:    0.3s finished
[Parallel(n_jobs=3)]: Using backend ThreadingBackend with 3 concurrent worker

3 / 125


...

[Parallel(n_jobs=3)]: Using backend ThreadingBackend with 3 concurrent workers.
[Parallel(n_jobs=3)]: Done  44 tasks      | elapsed:    0.0s
[Parallel(n_jobs=3)]: Done 100 out of 100 | elapsed:    0.1s finished
[Parallel(n_jobs=3)]: Using backend ThreadingBackend with 3 concurrent workers.
[Parallel(n_jobs=3)]: Done  44 tasks      | elapsed:    0.0s
[Parallel(n_jobs=3)]: Done 100 out of 100 | elapsed:    0.1s finished
[Parallel(n_jobs=3)]: Using backend LokyBackend with 3 concurrent workers.
[Parallel(n_jobs=3)]: Done   3 out of   3 | elapsed:    0.4s finished
[Parallel(n_jobs=3)]: Using backend ThreadingBackend with 3 concurrent workers.
[Parallel(n_jobs=3)]: Done  44 tasks      | elapsed:    0.0s
[Parallel(n_jobs=3)]: Done 100 out of 100 | elapsed:    0.1s finished
[Parallel(n_jobs=3)]: Using backend ThreadingBackend with 3 concurrent workers.
[Parallel(n_jobs=3)]: Done  44 tasks      | elapsed:    0.0s
[Parallel(n_jobs=3)]: Done 100 out of 100 | elapsed:    0.0s finished
[Parallel(n

113 / 125
114 / 125
115 / 125
116 / 125


[Parallel(n_jobs=3)]: Done  44 tasks      | elapsed:    0.0s
[Parallel(n_jobs=3)]: Done 100 out of 100 | elapsed:    0.0s finished
[Parallel(n_jobs=3)]: Using backend ThreadingBackend with 3 concurrent workers.
[Parallel(n_jobs=3)]: Done  44 tasks      | elapsed:    0.0s
[Parallel(n_jobs=3)]: Done 100 out of 100 | elapsed:    0.0s finished
[Parallel(n_jobs=3)]: Using backend LokyBackend with 3 concurrent workers.
[Parallel(n_jobs=3)]: Done   3 out of   3 | elapsed:    0.3s finished
[Parallel(n_jobs=3)]: Using backend ThreadingBackend with 3 concurrent workers.
[Parallel(n_jobs=3)]: Done  44 tasks      | elapsed:    0.0s
[Parallel(n_jobs=3)]: Done 100 out of 100 | elapsed:    0.0s finished
[Parallel(n_jobs=3)]: Using backend ThreadingBackend with 3 concurrent workers.
[Parallel(n_jobs=3)]: Done  44 tasks      | elapsed:    0.0s
[Parallel(n_jobs=3)]: Done 100 out of 100 | elapsed:    0.0s finished
[Parallel(n_jobs=3)]: Using backend LokyBackend with 3 concurrent workers.


117 / 125
118 / 125
119 / 125
120 / 125
121 / 125
122 / 125
123 / 125
124 / 125
125 / 125


[Parallel(n_jobs=3)]: Done   3 out of   3 | elapsed:    0.3s finished


In [54]:
scores_rf_pred_df = pd.DataFrame(scores_rf_predominance, columns=['hgb_weight', 'rf_weight', 'et_weight', 'bg_weight', 'score_soft', 'score_hard'])
scores_rf_pred_df.sort_values(by=["score_soft"])


,hgb_weight,rf_weight,et_weight,bg_weight,score_soft,score_hard
14,0.100,0.70,0.2000,0.0000,0.772505,0.759115
19,0.100,0.80,0.0575,0.0425,0.772788,0.759115
17,0.100,0.75,0.1050,0.0450,0.772838,0.759115
18,0.100,0.80,0.0100,0.0900,0.772949,0.759115
13,0.100,0.70,0.1525,0.0475,0.773336,0.759115
...,...,...,...,...,...,...
59,0.175,0.60,0.2000,0.0250,0.785311,0.759115
78,0.200,0.70,0.0100,0.0900,0.785394,0.759115
79,0.200,0.70,0.0575,0.0425,0.785685,0.759115
76,0.200,0.65,0.0575,0.0925,0.786689,0.759115


In [65]:
def get_harmonic_weight(model):
    return 1 / (1 - model.validation_score) ** 2
    
models = [hgb, rf, et, bg]
weights = [get_harmonic_weight(mod) for mod in [hgb, rf, et, bg]]

y_validation_pred_soft = vote(
        models=models, 
        weights=weights,
        policy='soft',
        kind='validation'
)
score_soft = custom_score(y_validation_pred_soft, y_validation_true)

y_validation_pred_hard = vote(
    models=models, 
    weights=weights, 
    policy='hard',
    kind='validation'
)
score_hard = custom_score(y_validation_pred_hard, y_validation_true)

print("weights :", weights)
print('soft :', score_soft)
print('hard :', score_hard)

[Parallel(n_jobs=3)]: Using backend ThreadingBackend with 3 concurrent workers.
[Parallel(n_jobs=3)]: Done  44 tasks      | elapsed:    0.0s
[Parallel(n_jobs=3)]: Done 100 out of 100 | elapsed:    0.0s finished
[Parallel(n_jobs=3)]: Using backend ThreadingBackend with 3 concurrent workers.
[Parallel(n_jobs=3)]: Done  44 tasks      | elapsed:    0.0s
[Parallel(n_jobs=3)]: Done 100 out of 100 | elapsed:    0.0s finished
[Parallel(n_jobs=3)]: Using backend LokyBackend with 3 concurrent workers.
[Parallel(n_jobs=3)]: Done   3 out of   3 | elapsed:    0.3s finished
[Parallel(n_jobs=3)]: Using backend ThreadingBackend with 3 concurrent workers.
[Parallel(n_jobs=3)]: Done  44 tasks      | elapsed:    0.0s
[Parallel(n_jobs=3)]: Done 100 out of 100 | elapsed:    0.0s finished
[Parallel(n_jobs=3)]: Using backend ThreadingBackend with 3 concurrent workers.
[Parallel(n_jobs=3)]: Done  44 tasks      | elapsed:    0.0s
[Parallel(n_jobs=3)]: Done 100 out of 100 | elapsed:    0.0s finished
[Parallel(n

soft : 0.79325345621389
hard : 0.7815097746360565


[Parallel(n_jobs=3)]: Done   3 out of   3 | elapsed:    0.3s finished


In [72]:
models = [hgb, rf, et, bg]
weights = [1.9, 1, 0.5, 0.5]

y_validation_pred_soft = vote(
        models=models, 
        weights=weights,
        policy='soft',
        kind='validation'
)
score_soft = custom_score(y_validation_pred_soft, y_validation_true)

y_validation_pred_hard = vote(
    models=models, 
    weights=weights, 
    policy='hard',
    kind='validation'
)
score_hard = custom_score(y_validation_pred_hard, y_validation_true)

print("weights :", weights)
print('soft :', score_soft)
print('hard :', score_hard)

[Parallel(n_jobs=3)]: Using backend ThreadingBackend with 3 concurrent workers.
[Parallel(n_jobs=3)]: Done  44 tasks      | elapsed:    0.0s
[Parallel(n_jobs=3)]: Done 100 out of 100 | elapsed:    0.1s finished
[Parallel(n_jobs=3)]: Using backend ThreadingBackend with 3 concurrent workers.
[Parallel(n_jobs=3)]: Done  44 tasks      | elapsed:    0.0s
[Parallel(n_jobs=3)]: Done 100 out of 100 | elapsed:    0.1s finished
[Parallel(n_jobs=3)]: Using backend LokyBackend with 3 concurrent workers.
[Parallel(n_jobs=3)]: Done   3 out of   3 | elapsed:    3.3s finished
[Parallel(n_jobs=3)]: Using backend ThreadingBackend with 3 concurrent workers.
[Parallel(n_jobs=3)]: Done  44 tasks      | elapsed:    0.0s
[Parallel(n_jobs=3)]: Done 100 out of 100 | elapsed:    0.1s finished
[Parallel(n_jobs=3)]: Using backend ThreadingBackend with 3 concurrent workers.
[Parallel(n_jobs=3)]: Done  44 tasks      | elapsed:    0.0s
[Parallel(n_jobs=3)]: Done 100 out of 100 | elapsed:    0.0s finished
[Parallel(n

weights : [1.9, 1, 0.5, 0.5]
soft : 0.7940404713195416
hard : 0.7885408477636184


[Parallel(n_jobs=3)]: Done   3 out of   3 | elapsed:    0.3s finished


In [73]:
# from kaggle_submit import submit_to_kaggle

# y_test_rf_advantaged = vote(models=[hgb, rf], weights=[0.25, 0.75], policy='soft', kind='test')
# y_test_rf_advantaged = y_test_rf_advantaged.astype(int)

# submit
# submit_to_kaggle(y_test_rf_advantaged, h5_test, fname="final_model_rf_advantaged.csv", msg="")

In [57]:
# y_test_hgb_advantaged = vote(models=[hgb, rf], weights=[0.75, 0.25], policy='soft', kind='test')
# y_test_hgb_advantaged = y_test_hgb_advantaged.astype(int)

# submit
# submit_to_kaggle(y_test_hgb_advantaged, h5_test, fname="final_model_rf_advantaged.csv", msg="")